<a href="https://colab.research.google.com/github/JASalles/Case-Biometria-Ifood/blob/main/Case_Biometria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação de Tabelas

In [56]:
import pandas as pd
import sqlite3
from google.colab import files

In [57]:
biometry = pd.read_csv('/content/drive/MyDrive/Case Ifood Biometria /biometry.csv', sep=';')
biometry_ex = pd.read_csv('/content/drive/MyDrive/Case Ifood Biometria /biometry_execution.csv', sep=';')
drivers = pd.read_csv('/content/drive/MyDrive/Case Ifood Biometria /drivers.csv', sep=';')
orders = pd.read_csv('/content/drive/MyDrive/Case Ifood Biometria /orders.csv', sep=';')

Tratamento biometry_ex

In [58]:
biometry_ex_tratada1 = biometry_ex[~((biometry_ex['Status'] == 'MATCH') & ((biometry_ex['Similarity'] == '0.100') | (biometry_ex['Similarity'] == 'NOT_MATCH')))]

biometry_ex_tratada = biometry_ex_tratada1.drop_duplicates()
#display(biometry_ex_tratada)

Tratamendo biometry

In [59]:
biometry_ex_aux = biometry_ex[(biometry_ex['Status'] == 'MATCH') & ((biometry_ex['Similarity'] == '0.100') | (biometry_ex['Similarity'] == 'NOT_MATCH'))]

#display(biometry_ex_aux)

In [60]:
biometry_tratada = biometry[(~biometry['Session_ID'].isin(biometry_ex_aux['Session_ID']))]

biometry_tratada = biometry_tratada.drop_duplicates()

#display(biometry_tratada)

In [61]:
orders_tratada = orders.drop_duplicates()

#display(orders_tratada)

In [62]:
drivers_tratada = drivers.drop_duplicates()

#display(drivers_tratada)

In [63]:
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

biometry

In [64]:
cursor.execute('''
CREATE TABLE biometry (
    Session_ID OBJECT,
    Event_Dt OBJECT,
    Status OBJECT,
    Driver_ID OBJECT,
    Action OBJECT
)
''')

In [65]:
biometry_tratada.to_sql('biometry', conn, if_exists='append', index=False)

471

In [66]:

query = "SELECT * FROM biometry"
df = pd.read_sql_query(query, conn)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471 entries, 0 to 470
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Session_ID  471 non-null    int64 
 1   Event_Dt    471 non-null    object
 2   Status      471 non-null    object
 3   Driver_ID   471 non-null    int64 
 4   Action      471 non-null    object
dtypes: int64(2), object(3)
memory usage: 18.5+ KB


In [67]:
#from IPython.display import display

#query = "SELECT * FROM biometry"
#df = pd.read_sql_query(query, conn)

#display(df)  # Mostra o DataFrame completo de forma formatada

biometry_ex

In [68]:
cursor.execute('''
CREATE TABLE biometry_ex (
    Event_Dt OBJECT,
    Session_ID OBJECT,
    Similarity FLOAT,
    Status OBJECT,
    Provider OBJECT
)
''')

In [69]:
biometry_ex_tratada.to_sql('biometry_ex', conn, if_exists='append', index=False)

520

In [70]:
query = "SELECT * FROM biometry_ex"
df = pd.read_sql_query(query, conn)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Event_Dt    520 non-null    object 
 1   Session_ID  520 non-null    int64  
 2   Similarity  469 non-null    float64
 3   Status      520 non-null    object 
 4   Provider    520 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 20.4+ KB


orders

In [71]:
cursor.execute('''
CREATE TABLE orders (
    Order_Dt OBJECT,
    Order_ID OBJECT,
    Driver_ID OBJECT,
    Order_Value FLOAT,
    Order_Status OBJECT
)
''')

In [72]:
orders_tratada.to_sql('orders', conn, if_exists='append', index=False)

9747

In [73]:
query = "SELECT * FROM orders"
df = pd.read_sql_query(query, conn)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9747 entries, 0 to 9746
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Order_Dt      9747 non-null   object 
 1   Order_ID      9747 non-null   int64  
 2   Driver_ID     9747 non-null   object 
 3   Order_Value   9747 non-null   float64
 4   Order_Status  9747 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 380.9+ KB


drivers

In [74]:
cursor.execute('''
CREATE TABLE drivers (
    Driver_ID OBJECT,
    Category OBJECT,
    Modal OBJECT,
    Register_Dt OBJECT,
    Device_ID OBJECT
)
''')

In [75]:
drivers_tratada.to_sql('drivers', conn, if_exists='append', index=False)

700

In [76]:
query = "SELECT * FROM drivers"
df = pd.read_sql_query(query, conn)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Driver_ID    700 non-null    int64 
 1   Category     700 non-null    object
 2   Modal        700 non-null    object
 3   Register_Dt  700 non-null    object
 4   Device_ID    700 non-null    object
dtypes: int64(1), object(4)
memory usage: 27.5+ KB


# A. Qual dos serviços mais falha?

(status = PROVIDER_FAILED)



## Visão por volumetria

In [77]:
result_a = """
SELECT
  Provider,
  COUNT(DISTINCT Session_ID) AS Falhas
FROM biometry_ex
WHERE Status = 'PROVIDER_FAILED'
GROUP BY Provider
ORDER BY Falhas DESC
-- LIMIT 1
"""

df_a = pd.read_sql_query(result_a, conn)
display(df_a)

,Provider,Falhas
0,A,40
1,B,8
2,C,3


## Visão por taxa de falhas

In [78]:
result_a_taxa = """
WITH agrupado AS (
  SELECT
    Provider,
    SUM(CASE WHEN Status = 'PROVIDER_FAILED' THEN 1 ELSE 0 END) AS Falhas,
    COUNT(DISTINCT Session_ID) AS Total_Sessoes
  FROM biometry_ex
  GROUP BY Provider
)
SELECT *,
  ROUND(Falhas * 100 / Total_Sessoes, 1) AS Taxa_Falhas
FROM agrupado
ORDER BY Taxa_Falhas DESC;

"""

df_a_taxa = pd.read_sql_query(result_a_taxa, conn)
display(df_a_taxa)

,Provider,Falhas,Total_Sessoes,Taxa_Falhas
0,C,3,9,33.0
1,B,8,40,20.0
2,A,40,471,8.0


# B. Qual categoria de entregador possui o maior índice falha na identificação biométrica?

(status = NOT_MATCH)

In [79]:
result_b = """
WITH agrupado AS (
  SELECT
    d.Category,
    SUM(CASE WHEN b.Status = 'NOT_MATCH' THEN 1 ELSE 0 END) AS Falhas,
    COUNT(DISTINCT b.Session_ID) AS Total_Sessoes
  FROM biometry AS b
  LEFT JOIN drivers AS d
    ON b.Driver_ID = d.Driver_ID
  GROUP BY d.Category
)
SELECT
  *,
  ROUND(Falhas * 100.0 / Total_Sessoes, 1) AS Taxa_Falhas
FROM agrupado
ORDER BY Taxa_Falhas DESC;

"""

df_b = pd.read_sql_query(result_b, conn)
display(df_b)

,Category,Falhas,Total_Sessoes,Taxa_Falhas
0,Bronze,25,141,17.7
1,Prata,20,144,13.9
2,Iniciante,11,92,12.0
3,Ouro,4,72,5.6
4,Diamante,1,22,4.5


# C. Calcule quanto o índice geral de MATCH seria se aumentássemos a similaridade mínima do MATCH para 0.90

In [80]:
result_c = """
WITH status_new AS (
  SELECT
    Session_ID,
    Status,
    CASE
      WHEN Similarity >= 0.9 THEN 'MATCH'
      WHEN Similarity < 0.9 THEN 'NOT_MATCH'
      WHEN Status = 'PROVIDER_FAILED' THEN 'PROVIDER_FAILED'
      END AS Status_new
  FROM biometry_ex
)

SELECT
  ROUND(SUM(CASE WHEN Status = 'MATCH' THEN 1 ELSE 0 END) * 100 / COUNT(Session_ID),1) AS Indice_Match_Status,
  ROUND(SUM(CASE WHEN Status_new = 'MATCH' THEN 1 ELSE 0 END) * 100 / COUNT(Session_ID),1) AS Indice_Match_Status_New
FROM status_new;
"""
df_c = pd.read_sql_query(result_c, conn)
display(df_c)

,Indice_Match_Status,Indice_Match_Status_New
0,78.0,59.0


# D. Você diria que há alguma relação entre volume de pedidos cancelados (status = CANCELLED) de um entregador e o status final do processo de identificação biométrica dele? Justifique sua resposta.

In [81]:
# Verificação da base geral

result_d_aux = """
SELECT
  *,
  DATE(b.Event_Dt),
  CASE
    WHEN Event_Dt <= Order_Dt THEN "1"
    ELSE 0
  END AS Biometria_Realizada,
  CASE
    WHEN Status == 'MATCH' THEN "1"
    ELSE "0"
  END AS Match,
  CASE
    WHEN Status == 'NOT_MATCH' THEN 1
    ELSE 0
  END AS Not_Match,
  CASE
    WHEN Status == 'PROVIDER_FAILED' THEN 1
    ELSE 0
  END AS Provider_Failed,
  CASE
    WHEN Order_Status == 'CANCELLED' THEN 1
    ELSE 0
  END AS Cancelados
FROM biometry b
LEFT JOIN orders o
    ON b.Driver_ID = o.Driver_ID
WHERE DATE(b.Event_Dt) = '2021-08-01'
AND Status == 'NOT_MATCH'
ORDER BY Status DESC

"""

df_d_aux = pd.read_sql_query(result_d_aux, conn)
display(df_d_aux)

,Session_ID,Event_Dt,Status,Driver_ID,Action,Order_Dt,Order_ID,Driver_ID,Order_Value,Order_Status,DATE(b.Event_Dt),Biometria_Realizada,Match,Not_Match,Provider_Failed,Cancelados
0,33891330,2021-08-01,NOT_MATCH,9009550,OK,2021-08-18,496704109813,9009550,36.25,CONCLUDED,2021-08-01,1,0,1,0,0
1,33891330,2021-08-01,NOT_MATCH,9009550,OK,2021-08-19,80334049004,9009550,106.59,CONCLUDED,2021-08-01,1,0,1,0,0
2,33891330,2021-08-01,NOT_MATCH,9009550,OK,2021-08-20,480966801046,9009550,28.25,CONCLUDED,2021-08-01,1,0,1,0,0
3,33891330,2021-08-01,NOT_MATCH,9009550,OK,2021-08-20,595057278109,9009550,16.45,CONCLUDED,2021-08-01,1,0,1,0,0
4,33891330,2021-08-01,NOT_MATCH,9009550,OK,2021-08-20,677999930401,9009550,103.16,CANCELLED,2021-08-01,1,0,1,0,1
5,33891330,2021-08-01,NOT_MATCH,9009550,OK,2021-08-21,573990713939,9009550,22.52,CONCLUDED,2021-08-01,1,0,1,0,0
6,33891330,2021-08-01,NOT_MATCH,9009550,OK,2021-08-21,649259924885,9009550,41.67,CONCLUDED,2021-08-01,1,0,1,0,0
7,33891330,2021-08-01,NOT_MATCH,9009550,OK,2021-08-22,401177890260,9009550,108.78,CONCLUDED,2021-08-01,1,0,1,0,0
8,33891330,2021-08-01,NOT_MATCH,9009550,OK,2021-08-23,814512949146,9009550,23.02,CONCLUDED,2021-08-01,1,0,1,0,0
9,33891330,2021-08-01,NOT_MATCH,9009550,OK,2021-08-23,975836536165,9009550,80.71,CONCLUDED,2021-08-01,1,0,1,0,0


In [82]:
result_d = """
WITH validacao_bio AS (
SELECT
  b.Event_Dt,
  o.Order_ID,
  b.Driver_ID,
  CASE
    WHEN Event_Dt <= Order_Dt THEN "1"
    ELSE 0
  END AS Biometria_Realizada,
  CASE
    WHEN Status == 'MATCH' THEN "1"
    ELSE "0"
  END AS Match,
  CASE
    WHEN Status == 'NOT_MATCH' THEN 1
    ELSE 0
  END AS Not_Match,
  CASE
    WHEN Status == 'PROVIDER_FAILED' THEN 1
    ELSE 0
  END AS Provider_Failed,
  CASE
    WHEN Order_Status == 'CANCELLED' THEN 1
    ELSE 0
  END AS Cancelados
FROM biometry b
LEFT JOIN orders o
    ON b.Driver_ID = o.Driver_ID
)
SELECT
  DATE(Event_Dt) AS Event_Dt,
  COUNT(DISTINCT Order_ID) AS Total_Pedidos,
  SUM(Match) AS Match,
  SUM(Not_Match) AS Not_Match,
  SUM(Provider_Failed) AS Provider_Failed,
  SUM(Cancelados) AS Cancelados
FROM validacao_bio
WHERE Biometria_Realizada == "1"
GROUP BY DATE(Event_Dt)
ORDER BY DATE(Event_Dt)

"""

df_d = pd.read_sql_query(result_d, conn)
display(df_d)

,Event_Dt,Total_Pedidos,Match,Not_Match,Provider_Failed,Cancelados
0,2021-08-01,126,108,18,0,36
1,2021-08-02,187,152,35,0,19
2,2021-08-03,131,126,5,0,21
3,2021-08-04,241,236,5,0,43
4,2021-08-05,196,196,0,0,38
5,2021-08-06,204,204,0,0,31
6,2021-08-07,54,34,17,3,13
7,2021-08-08,244,194,23,27,25
8,2021-08-09,202,198,0,4,26
9,2021-08-10,175,168,7,0,32


# E. Em quais dias provavelmente tivemos um aumento do comportamento indevido “empréstimo de conta”?

- Análise de sessões e drivers para verificar se cada entregador tem apenas uma sessão

- not_mach + ok

- match + fluxo stack

- PROVIDER_FAILED + ok

- match + cancelado

- Quantidade de entregas por driver no dia

In [83]:
# Verificação da quantidade de sessões por driver na base final de biometria

result_e_a = """
SELECT
  Event_Dt,
  Total_Sessoes_Distintos,
  Total_Drivers_Distintos
FROM (
    SELECT
      DATE(Event_Dt) AS Event_Dt,
      COUNT(DISTINCT Session_ID) AS Total_Sessoes_Distintos,
      COUNT(DISTINCT Driver_ID) AS Total_Drivers_Distintos
    FROM biometry
    GROUP BY DATE(Event_Dt)
) AS aux
WHERE Total_Sessoes_Distintos != Total_Drivers_Distintos
ORDER BY Total_Sessoes_Distintos DESC;
"""

df_e_a = pd.read_sql_query(result_e_a, conn)
display(df_e_a)

,Event_Dt,Total_Sessoes_Distintos,Total_Drivers_Distintos


In [84]:
result_e_b = """
WITH geral AS (
SELECT
  DATE(Event_Dt) AS Event_Dt,
  CASE
    WHEN Status == 'MATCH' AND Action == "FLUXO_STACK" THEN "1"
    ELSE "0"
  END AS Match_FS,
  CASE
    WHEN Status == 'NOT_MATCH' AND Action == "OK" THEN 1
    ELSE 0
  END AS Not_Match_Liberado,
  CASE
    WHEN Status == 'PROVIDER_FAILED' AND Action == "OK" THEN 1
    ELSE 0
  END AS Falha_Liberado
FROM biometry
)
SELECT
  Event_Dt,
  SUM(Match_FS) AS Match_FS,
  SUM(Not_Match_Liberado) AS Not_Match_Liberado,
  SUM(Falha_Liberado) AS Falha_Liberado,
  SUM(Match_FS) + SUM(Not_Match_Liberado) + SUM(Falha_Liberado) AS Total_Inconsistencias
FROM geral
GROUP BY Event_Dt
ORDER BY Event_Dt
"""

df_e_b = pd.read_sql_query(result_e_b, conn)
display(df_e_b)

,Event_Dt,Match_FS,Not_Match_Liberado,Falha_Liberado,Total_Inconsistencias
0,2021-08-01,1,1,0,2
1,2021-08-02,2,2,0,4
2,2021-08-03,1,1,0,2
3,2021-08-04,1,1,0,2
4,2021-08-05,0,0,0,0
5,2021-08-06,0,0,0,0
6,2021-08-07,1,1,0,2
7,2021-08-08,2,1,1,4
8,2021-08-09,0,0,0,0
9,2021-08-10,1,1,0,2


In [85]:
# Match + Pedido Cancelado

result_e_c = """
WITH validacao_match AS (
SELECT
  Event_Dt,
  Order_ID,
  CASE
    WHEN Event_Dt <= Order_Dt THEN "1"
    ELSE 0
  END AS Biometria_Realizada,
  CASE
    WHEN Status == 'MATCH' AND Order_Status == 'CANCELLED'  THEN "1"
    ELSE "0"
  END AS Match_Cancelado
FROM biometry b
LEFT JOIN orders o
    ON b.Driver_ID = o.Driver_ID
)
SELECT
  DATE(Event_Dt) AS Event_Dt,
  COUNT(DISTINCT Order_ID) AS Total_Pedidos,
  SUM(Match_Cancelado) AS Match_Cancelado
FROM validacao_match
WHERE Biometria_Realizada == "1"
GROUP BY DATE(Event_Dt)
ORDER BY DATE(Event_Dt)

"""

df_e_c = pd.read_sql_query(result_e_c, conn)
display(df_e_c)

,Event_Dt,Total_Pedidos,Match_Cancelado
0,2021-08-01,126,30
1,2021-08-02,187,13
2,2021-08-03,131,20
3,2021-08-04,241,42
4,2021-08-05,196,38
5,2021-08-06,204,31
6,2021-08-07,54,6
7,2021-08-08,244,16
8,2021-08-09,202,24
9,2021-08-10,175,30


In [86]:
# Volume de pedidos

result_e_d = """
WITH pedidos_por_entregador AS (
SELECT
  Driver_ID,
  COUNT(Order_ID) AS Total_Pedidos
FROM orders
WHERE Driver_ID != '#N/D'
GROUP BY Driver_ID
),
media_geral AS (
SELECT
  AVG(Total_Pedidos) AS Media_Total_Pedidos
FROM pedidos_por_entregador
)
SELECT
  p.Driver_ID,
  p.Total_Pedidos,
  m.Media_Total_Pedidos,
  CASE
    WHEN p.Total_Pedidos > m.Media_Total_Pedidos THEN 1
    ELSE 0
  END AS Acima_Media
FROM pedidos_por_entregador p
CROSS JOIN media_geral m
ORDER BY p.Total_Pedidos DESC

"""
df_e_d = pd.read_sql_query(result_e_d, conn)
display(df_e_d)

,Driver_ID,Total_Pedidos,Media_Total_Pedidos,Acima_Media
0,6140918,48,13.914286,1
1,3552604,43,13.914286,1
2,4123969,34,13.914286,1
3,7733668,34,13.914286,1
4,3661602,33,13.914286,1
...,...,...,...,...
695,4998025,2,13.914286,0
696,6247294,2,13.914286,0
697,7011119,2,13.914286,0
698,3753174,1,13.914286,0


In [87]:
result_e_d = """
WITH pedidos_por_entregador AS (
SELECT
  Driver_ID,
  COUNT(Order_ID) AS Total_Pedidos
FROM orders
WHERE Driver_ID != '#N/D'
GROUP BY Driver_ID
),
media_geral AS (
SELECT
  AVG(Total_Pedidos) AS Media_Total_Pedidos
  FROM pedidos_por_entregador
),
acima_media_final AS (
SELECT
  p.Driver_ID,
  p.Total_Pedidos,
  m.Media_Total_Pedidos,
  CASE
    WHEN p.Total_Pedidos > m.Media_Total_Pedidos THEN 1
    ELSE 0
  END AS Acima_Media
FROM pedidos_por_entregador p
CROSS JOIN media_geral m
)
SELECT
  ROUND(SUM(Acima_Media) * 100 / COUNT(*), 1) AS Percentual_Acima_Media
FROM acima_media_final;
"""

df_e_d = pd.read_sql_query(result_e_d, conn)
display(df_e_d)

,Percentual_Acima_Media
0,49.0
